In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import re, random, sys, pickle
                                            # tfidf          # bad-of-words
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, f1_score

import nltk
from nltk.corpus import wordnet

sys.path.insert(0,'..')
from fields import *


In [2]:
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [2]:
sheet = pd.read_excel('../sheets/DRE_07.2022.xlsx')

# Model training
Sheet theme recognizer

## Generate data

In [13]:
def find_synonyms(words, qty):

    new_words = []
    for word in words:
        if len(word.split(' ')) > 1:
            word = word.split(' ')[0]

        synonyms=[];
        for syn in wordnet.synsets(word, lang='por'):
            for lemma in syn.lemmas(lang='por'):
                synonyms.append(lemma.name())

        if len(synonyms) > 0:
            new_words.append(np.random.choice(synonyms, size=qty)[0])
        else:
            new_words.append(word)

    return new_words

def vectorize_csv(df, vectorizer, only_strings=False):
    
    if isinstance(df, np.ndarray): df = pd.DataFrame(df)

    if only_strings is True:
        is_word = r'[A-Za-z\s]+'
        words = []
        for column in df.select_dtypes(include=['object']):
            for cell in df[column]:
                if isinstance(cell, str):
                    matches = re.findall(is_word, cell)
                    words.extend(matches)
        text = ' '.join(words)
    else:
        flatten = df.astype(str).values.flatten()
        text = ' '.join(flatten)
    # Fit the vectorizer with a 1-D array, only if it has not been fitted before
    if not hasattr(vectorizer, 'vocabulary_'):
        vectorizer.fit([text])
    # Transforming text placed in a 2-D array using the transform method
    vector=[]
    display(text)
    # vectorize many
    if len(df.columns) == 1:
        for doc in df.iloc[:, 0]:
            vec = vectorizer.transform([doc]).toarray()
            vector.extend(vec)
    # vectorize only one document
    else:
        vec = vectorizer.transform([text]).toarray()
        vector.extend(vec)
    vector=np.array(vector)

    if not hasattr(vectorizer, 'vocabulary_'):
        return vector, vectorizer
    return vector


def adjust_vectors(vectors, max_=None):

    adjusted_vectors = []
    if max_ is None:
        lengths = [len(vec) for vec in vectors]
        max_ = np.max(lengths)

    for vec in vectors:
        fill = max_ - len(vec)
        if fill > 0:
            mode = stats.mode(vec, keepdims=True)[0][0]
            fill = np.full(fill, mode)
            vec = np.concatenate([vec, fill], axis=0)
            adjusted_vectors.append(vec)
        else:
            adjusted_vectors.append(vec)

    return np.array(adjusted_vectors)


def generate_accounting_data(qty, type_, how, vectorizer=None):
    if type_ in ["dre", 1]: fields, label = dre, 1
    if type_ in ["balancete", 2]: fields, label = balancete, 2

    data = []
    size = int( len(list(fields.keys())) * random.uniform(0.6, 1) )
    for i in range(0, qty):
        dfs = []; text = []
        
        random_fields = np.random.choice(list(fields.keys()), size)
        for f in random_fields:
            words = [f, *fields[f]]
            words = find_synonyms(words, 1)

            if how=="text":
                text.extend(words)

            elif how=="sheet":
                values = [round(val, 2) for val in np.random.uniform(1e5, 2e5, size=len(words))]
                df = pd.DataFrame(data=[words, values]).T
                dfs.append(df)
        
        if how=="text":
            data.append(' '.join(text))

        elif how=="sheet":
            dfs = pd.concat(dfs)
    
            # data.append(dfs)
    X = np.array(data); y = np.full(len(data), label)

    if vectorizer:
        print(vectorizer)
        # vectorize words by row
        # vectors = []
        # for row in X:
        X = vectorize_csv(X, vectorizer, True)
            # vectors.append(vector)
        # adjusting shape
        # display(vectors.shape, vectors)
        # adjusted_vectors = adjust_vectors(vectors)
        return X, y, vectorizer
    
    return X, y

# X_, y_ = generate_accounting_data(10, 'dre', 'text', TfidfVectorizer())
X=[]; y=[]; vectorizers = {}
for doc in ['dre', 'balancete']:
    X_, y_, vectorizer = generate_accounting_data(50, doc, 'text', CountVectorizer())
    vectorizers[doc] = vectorizer
    X.extend(X_); y.extend(y_)

X=adjust_vectors(X); y=np.array(y)
X.shape, y.shape

CountVectorizer()


'conclus o Receitas custos despesas resultado resultado Dedu es Devolu es imposto EBITDA benef cio deprecia o amortiza o custo Mat ria prima for a de trabalho custos deprecia o continua o EBITDA vantagem deprecia o Amortiza o resultado conseq ncia rendimento Receitas despesas EBITDA handicap deprecia o amortiza o EBITDA handicap deprecia o Amortiza o Dedu es Devolu es imposto resultado Receitas pre o despesas fim efeitos fim juros Receitas despesas despesas Comerciais Administrativas Financeiras Dedu es Devolu es taxa Dedu es Devolu es colectas despesas Comerciais Administrativas Financeiras EBITDA benef cio deprecia o amortiza o despesas Comerciais Administrativas Financeiras Dedu es Devolu es taxa receita Vendas servi os Outros provento Vendas servi os Outros conseq ncia juro Receitas despesas Dedu es Devolu es colectas Dedu es Devolu es tributo fim Receitas pre o despesas continua o conseq ncia resultado Receitas despesas efeitos conseq ncia resultado conclus o Dedu es Devolu es imp

CountVectorizer()


'ativo Circulante realiz vel Investimentos Imobilizado Intang vel ativo Circulante que se pode alcan ar Investimentos Imobilizado Intang vel despesas custos custos despesas r pido Circulante que se pode alcan ar Investimentos Imobilizado Intang vel ativo Circulante conquist vel Investimentos Imobilizado Intang vel Receitas Vendas Vendas Outras despesas custo custo despesas Receitas Vendas Vendas Outras bem capital reservas Ajustes reservas ecologia responsabilidade Circulante Exig vel Receitas Vendas Vendas Outras despesas custos custo despesas l pido Circulante que se pode conquistar Investimentos Imobilizado Intang vel obriga o Circulante Exig vel Receitas Vendas Vendas Outras despesas pre o custo despesas Patrim nio Capital reservas ecologia  Ajustes reservas ecologia  passivo Circulante Exig vel patrim nio fundo reservas Ajustes reservas Receitas Vendas Vendas Outras heran a Capital reserva Ajustes reserva d vida Circulante Exig vel obriga o Circulante Exig vel despesas custo pre o

((100, 83), (100,))

## Training model

In [14]:
svm = SVC(kernel='linear', probability=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print( classification_report(y_test,y_pred) )

(80, 83)
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



## Real testing

In [8]:
# hasattr(CountVectorizer(), 'vocabulary_'), hasattr(vectorizers['dre'], 'vocabulary_')
def clean_sheet(sheet):
    #clean nan cols and rows
    sheet = sheet.dropna(how='all', axis=1).dropna(how='all', axis=0)
    #most frequent row nans pattern
    mode = stats.mode(sheet.isna().sum(axis=1), keepdims=True)[0][0]
    #assuming this as where to scrap data
    data = pd.concat([row for _, row in sheet.iterrows() if row.isna().sum() == mode], axis=1).T

    valid_data = pd.concat(
        [
            col for _, col in data.items() 
            if col.isna().sum() < len(col)*0.1
        ], 
    axis=1).reset_index()

    return valid_data

In [7]:
dre_test = pd.read_excel('../sheets/DRE_07.2022.xlsx', index_col=[0])
valid_data = clean_sheet(dre_test)
vector = vectorize_csv(valid_data, vectorizers['dre'], only_strings=True)
doc_type = svm.predict(vector)

NameError: name 'vectorize_csv' is not defined

In [10]:
balan_test = pd.read_excel('../sheets/Cópia BALANCETE_31.07.2022.xlsx', index_col=[0])
valid_data = clean_sheet(balan_test)
vector = vectorize_csv(valid_data, vectorizers['dre'], only_strings=True)
svm.predict(vector)

KeyError: 'dre'

In [155]:
pickle.dump(svm, open('../models/svm_1.pkl', 'wb'))


# Scraping

In [99]:
dre_test = pd.read_excel('../sheets/DRE_07.2022.xlsx', index_col=[0])
balan_test = pd.read_excel('../sheets/Cópia BALANCETE_31.07.2022.xlsx')

In [113]:
balan_test.head(10)
clean_sheet(balan_test)

,index,Unnamed: 0,Unnamed: 2,Unnamed: 4,Unnamed: 7,Unnamed: 9,Unnamed: 11,Unnamed: 13
0,5,Conta Contábil,Cod. R.,NaN,NaN,Débito,Crédito,S. Atual
1,7,1.0.00.00.00.000000,1,A T I V O,1899516.77,15189090.34,14995080.36,2093526.75
2,9,1.1.00.00.00.000000,11,ATIVO CIRCULANTE,1725485.06,15180120.94,14973085.5,1932520.5
3,11,1.1.01.00.00.000000,12,CAIXA E EQUIVALENTE DE CAIXA,384989.03,4965261.93,5273272.6,76978.36
4,13,1.1.01.01.00.000000,13,CAIXA GERAL,802.4,17445.66,16513.91,1734.15
...,...,...,...,...,...,...,...,...
354,446,5.1.90.01.00.000000,51061,OUTRAS RECEITAS OPERACIONAIS,0,0,102566.96,-102566.96
355,447,5.1.90.01.05.000000,529559,ICMS S/OUTRAS ENTRADAS,0,0,288,-288
356,448,5.1.90.01.06.000000,529562,IPI S/OUTRAS ENTRADAS,0,0,576,-576
357,449,5.1.90.01.00.000002,5922,RECUPERACAO CUSTOS/DESPESAS,0,0,4272.52,-4272.52


In [151]:
def data_scraping(sheet, doc_type):
    
    valid_data = clean_sheet(sheet).astype(str)
    
    # EXTRACT ACCOUNTING FIELDS
    fields = []
    strings = valid_data.applymap(lambda x: re.findall(r'\D+', x)) #[^0-9\.,]
    for _, col in strings.items():
        # check valid columns
        if col.tolist().count([]) < len(col):
            for row in col:
                if len(row) > 0:
                    fields.extend(row)

    # EXTRACT FIELDS VALUE
    all_values = []
    numbers = valid_data.applymap(lambda x: re.findall(r'\d+', x))
    for _, col in numbers.items():
        values = []
        # check valid columns
        if col.tolist().count([]) < len(col):
            for row in col:
                if len(row) > 1:
                    try:
                        decimal = float( int(row[-1])/100 )
                        integer = int(''.join(row[:-1]))
                        values.append(integer + decimal)
                    except:
                        pass
                else:
                    values.append(np.nan)
        if len(values) > 0:    
            all_values.append(values)

    if doc_type == 1:
        data = pd.DataFrame(data=[fields, *all_values]).T
    
    elif doc_type == 2:
        data = pd.DataFrame(data=[fields, *all_values]).T

    return data.dropna(how='all', axis=1)

# data_scraping(dre_test, 1).head(50)
data_scraping(balan_test, 2).head(50)

,0,2,4,5,6,7,8
0,Conta Contábil,NaN,NaN,NaN,NaN,NaN,NaN
1,.,10000000.0,NaN,1899516.77,15189090.34,14995080.36,2093526.75
2,.,11000000.0,NaN,1725485.06,15180120.94,14973085.05,1932520.05
3,.,11010000.0,NaN,384989.03,4965261.93,5273272.06,76978.36
4,.,11010100.0,NaN,802.04,17445.66,16513.91,1734.15
5,.,11010101.01,NaN,802.04,17445.66,16513.91,1734.15
6,.,11010200.0,NaN,293350.89,4584831.05,4859522.32,18659.62
7,.,11010201.0,NaN,293350.89,4584831.05,4859522.32,18659.62
8,.,11010201.04,NaN,293350.89,4107843.71,4382534.98,18659.62
9,.,11010201.01,NaN,NaN,476987.34,476987.34,NaN
